# RAG Evaluator

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxJudge/rag_evaluation.ipynb)

In [ ]:
!pip install indoxJudge -U
!pip install transformers
!pip install torch
!pip install openai

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indoxJudge`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indoxJudge
```
2. **Activate the virtual environment:**
```bash
indoxJudge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indoxJudge
```

2. **Activate the virtual environment:**
    ```bash
   source indoxJudge/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [3]:
query = "What are the benefits of diet?"
retrieval_context = ["The Mediterranean diet emphasizes eating primarily plant-based foods, such as fruits and vegetables, whole grains, legumes, and nuts. It also includes moderate amounts of fish and poultry, and low consumption of red meat. Olive oil is the main source of fat, providing monounsaturated fats which are beneficial for heart health.","Research has shown that the Mediterranean diet can reduce the risk of heart disease, stroke, and type 2 diabetes. It is also associated with improved cognitive function and a lower risk of Alzheimer's disease. The diet's high content of fiber, antioxidants, and healthy fats contributes to its numerous health benefits.","A Mediterranean diet has been linked to a longer lifespan and a reduced risk of chronic diseases. It promotes healthy aging and weight management due to its emphasis on whole, unprocessed foods and balanced nutrition."]
response = "The Mediterranean diet is known for its health benefits, including reducing the risk of heart disease, stroke, and diabetes. It encourages the consumption of fruits, vegetables, whole grains, nuts, and olive oil, while limiting red meat. Additionally, this diet has been associated with better cognitive function and a reduced risk of Alzheimer's disease, promoting longevity and overall well-being."

## Importing Required Modules
imports the necessary classes from the indoxJudge library. The RAGevaluator class is used for evaluating retrieval-augmented generation models, assessing their performance based on various metrics. Additionally, OpenAi is the class used to interact with OpenAI models, such as GPT-3.5, providing the foundational language model capabilities for evaluation.

In [4]:
from indoxJudge.pipelines import RagEvaluator
from indoxJudge.models import OpenAi

## Initializing the OpenAI Model
Here, the OpenAi class is instantiated to create a model object that interacts with OpenAI's gpt-3.5-turbo-0125 model. The api_key is passed to authenticate the API request. Replace OPENAI_API_KEY with your actual API key.

In [5]:
# Initialize the language model
# it can be any OpenAI model, please refer to the [OpenAI Models documentation](https://platform.openai.com/docs/models) such as GPT-4o. or other models

model = OpenAi(api_key=OPENAI_API_KEY,model="gpt-4o-mini")

INFO: Initializing OpenAi with model: gpt-4o-mini and max_tokens: 2048


The RAGevaluator class is instantiated to create an evaluator object. This object is configured to assess the model’s performance by using a specific response, retrieval context, and query. The llm_as_judge parameter is set to the model created in the previous step, while llm_response, retrieval_context, and query are additional components utilized during the evaluation process.

In [6]:
evaluator = RagEvaluator(llm_as_judge=model,llm_response=response,retrieval_context=retrieval_context,query=query)

**Explanation:**

- **Running the Evaluation:** This line calls the `judge` method on the `evaluator` object. The `judge` method runs through all the specified metrics (e.g., Faithfulness, Answer Relevancy, Hallucination, GEval, Knowledge Retention, BertScore, METEOR) to evaluate the language model's performance.
  
- **Logging the Process:** As the evaluation runs, the process logs the start and completion of each metric evaluation, providing feedback on the progress. Each log entry is tagged with an INFO level, indicating routine operational messages.
  
- **Handling Warnings:** You may notice a warning regarding model initialization and a future deprecation notice from the Hugging Face Transformers library. These warnings inform you about potential issues related to model compatibility and upcoming changes in the library.
  
- **Completion of Metrics:** After all metrics have been evaluated, the `judge` method completes, and the evaluation results are stored in the `eval_result` dictionary for further analysis or reporting.



In [7]:
eval_result = evaluator.judge()

INFO: Model set for all metrics.
INFO: RagEvaluator initialized with model and metrics.

INFO: Evaluating metric: Faithfulness
INFO: Token Counts - Input: 257 | Output: 172
INFO: Token Counts - Input: 222 | Output: 174
INFO: Token Counts - Input: 893 | Output: 181
INFO: Token Counts - Input: 240 | Output: 45
INFO: Token Usage Summary:
 Total Input: 1612 | Total Output: 572 | Total: 2184
INFO: Completed evaluation for metric: Faithfulness, score: 0.93

INFO: Evaluating metric: AnswerRelevancy
INFO: Token Counts - Input: 198 | Output: 153
INFO: Token Counts - Input: 524 | Output: 168
INFO: Token Counts - Input: 196 | Output: 35
INFO: Token Usage Summary:
 Total Input: 918 | Total Output: 356 | Total: 1274
INFO: Completed evaluation for metric: AnswerRelevancy, score: 1.0

INFO: Evaluating metric: ContextualRelevancy
INFO: Token Counts - Input: 346 | Output: 46
INFO: Token Counts - Input: 344 | Output: 36
INFO: Token Counts - Input: 322 | Output: 40
INFO: Token Counts - Input: 346 | Outpu

**Explanation:**

- **Retrieving Metric Scores:** The first line assigns the detailed metric scores from the evaluation to the variable `evaluator_metrics_score`. These scores reflect the performance of the language model across individual metrics such as Faithfulness, Answer Relevancy, Hallucination, GEval, Knowledge Retention, BertScore, METEOR, Recall, F1 Score.

- **Retrieving Overall Evaluation Score:** The second line assigns the overall evaluation score to the variable `evaluator_evaluation_score`. This score is typically a cumulative measure that reflects the model's performance across all evaluated metrics.

- **Example Output:** The dictionary shown represents a typical output of `evaluator_metrics_score`. Each key corresponds to a specific metric, and the associated value indicates the model's score for that metric. For instance, a `1.0` score in `AnswerRelevancy` and `KnowledgeRetention` indicates perfect performance in those areas, while other metrics like `METEOR` show a more moderate performance.


In [8]:
evaluator_metrics_score = evaluator.metrics_score

In [9]:
evaluator_metrics_score

{'Faithfulness': 0.93,
 'AnswerRelevancy': 1.0,
 'ContextualRelevancy': 1.0,
 'GEval': 0.8,
 'Hallucination': 0.0,
 'KnowledgeRetention': 1.0,
 'precision': 0.74,
 'recall': 0.77,
 'f1_score': 0.75,
 'METEOR': 0.76,
 'evaluation_score': 0.97}

**Explanation:**

- **Plotting Evaluation Metrics:** This line generates visual plots of the evaluation metrics using the `plot` method of the `evaluator` object. The plots provide a graphical representation of the model's performance across different metrics, making it easier to analyze and compare the results.

- **Dash UI Interface:** When `mode="external"` is used, this will open a Dash UI in a new browser window or tab to display the evaluation metrics plots interactively.

- **Colab Users:** If you are using Google Colab, it is recommended to set `mode="inline"` instead. This will render the plots directly within the notebook, making it more convenient for users working in an online environment like Colab.


In [10]:
evaluator.plot(mode="external",interpreter=model)

Dash app running on http://127.0.0.1:8050/
